### ⚠️ Note on Scope: U.S.-Only Implementation

This notebook demonstrates the complete data processing and analysis workflow for **the United States (USA)**.

While similar analyses have been conducted for **China** and **Europe**, only the U.S. version is included here for clarity and reproducibility.

✅ **The methodology and code structure are consistent across all regions.**  
The only differences lie in:
- Input data files (e.g., CSVs, shapefiles)
- Regional identifiers (e.g., county codes, administrative levels)
- Data sources and file paths

Therefore, this U.S. example serves as a **fully representative template** for understanding and replicating the analysis in other regions.

All steps — including data merging, normalization, city-level aggregation, and matching level calculation — follow the same logic globally.

# Maximum and minimum solar data loading in USA

In [ ]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
import rasterio

# File paths
tiff_file = 'USA_GISdata_LTAy_AvgDailyTotals_GlobalSolarAtlas-v2_GEOTIFF/western-hemisphere/GHI.tif'  # from External data
city_boundaries_path = 'merged_city_boundariesUSCounties2.shp'  # from External data
output_csv_file = 'US_cities_GHI_min_max1.csv'

# Load city boundary data, specifying utf-8 encoding to prevent garbled text
gdf_cities = gpd.read_file(city_boundaries_path, encoding='utf-8', engine='pyogrio')

# Ensure that the CRS of the city boundary data matches that of the raster data
# This step is necessary because zonal_stats requires vector and raster data to be in the same coordinate system
gdf_cities = gdf_cities.to_crs(rasterio.open(tiff_file).crs)

# Calculate the maximum and minimum values of GHI within each city area
print("Starting calculation of maximum and minimum GHI values within each city area...")
stats = zonal_stats(
    vectors=gdf_cities,
    raster=tiff_file,
    stats=['min', 'max'],
    nodata=-9999,  # Set no data value to -9999
    all_touched=True  # Include all pixels touched by polygons. By default False, considers only pixels whose center is within the polygon
)

# Add the results to the original GeoDataFrame
gdf_cities['GHI_min'] = [stat['min'] for stat in stats]
gdf_cities['GHI_max'] = [stat['max'] for stat in stats]  # Note: The original code had PVOUT_max here, but it's corrected to GHI_max as per the filename

# Retain only the required columns (assuming 'NAME' is the field name for city names, replace if different)
gdf_output = gdf_cities[['STATEFP', 'COUNTYFP', 'COUNTYNS', 'NAME', 'GHI_min', 'GHI_max']]

# Save to CSV file, specifying utf-8 encoding to ensure non-ASCII characters are stored correctly
gdf_output.to_csv(output_csv_file, index=False, encoding='utf-8')

print(f"Processed data has been saved as: '{output_csv_file}'")

In [ ]:
import pandas as pd

# File paths (please adjust according to your actual file locations)
charging_stations_file = 'filtered_USA_charging_stations.csv'
pvout_stats_file = 'US_cities_GHI_min_max1.csv'
output_csv_file = 'USAmatched_charging_stations_within_city_boundariesGHI_min_max.csv'

# Read the charging stations data
df_charging_stations = pd.read_csv(charging_stations_file)

# Read the PVOUT statistics data
df_pvout_stats = pd.read_csv(pvout_stats_file)

# Ensure the 'COUNTYNS' column in both DataFrames has the same data type to avoid merge issues due to type mismatch
df_charging_stations['COUNTYNS'] = df_charging_stations['COUNTYNS'].astype(str)
df_pvout_stats['COUNTYNS'] = df_pvout_stats['COUNTYNS'].astype(str)

# Merge the two DataFrames on the 'COUNTYNS' column
df_merged = pd.merge(df_charging_stations, df_pvout_stats[['COUNTYNS', 'GHI_min', 'GHI_max']], on='COUNTYNS', how='left')

# Save the merged result to a new CSV file
df_merged.to_csv(output_csv_file, index=False, encoding='utf-8')

print(f"Processed data has been saved as: '{output_csv_file}'")

# Print the first few rows to inspect the result
print("First few rows of the merged data:")
print(df_merged.head())

# Maximum and minimum wind energy loading in USA

In [ ]:
import geopandas as gpd
import pandas as pd

# Define file paths
csv_file_path = 'USAmatched_charging_stations_within_city_boundariesGHI_min_max.csv'
shapefile_ghi_path = 'USA_city_wind\\USA_city_wind.shp'  # from External data
output_csv_path = 'USAmatched_charging_stations_within_city_boundaries_GHIwind_maxmin.csv'

# Read the CSV file
df_charging_stations = pd.read_csv(csv_file_path)

# Ensure the COUNTYNS field is a fixed-length string (e.g., 8 digits)
df_charging_stations['COUNTYNS'] = df_charging_stations['COUNTYNS'].astype(str).str.zfill(8)

# Read the Shapefile and convert CRS if necessary
gdf_city_ghi = gpd.read_file(shapefile_ghi_path, engine='pyogrio')
gdf_city_ghi.to_crs(epsg=4326, inplace=True)

# Ensure the COUNTYNS field in the Shapefile is also a fixed-length string
gdf_city_ghi['COUNTYNS'] = gdf_city_ghi['COUNTYNS'].astype(str).str.zfill(8)

# Clean the COUNTYNS field by stripping any extra whitespace
df_charging_stations['COUNTYNS'] = df_charging_stations['COUNTYNS'].str.strip()
gdf_city_ghi['COUNTYNS'] = gdf_city_ghi['COUNTYNS'].str.strip()

# Rename columns in the Shapefile for clarity before merging
gdf_city_ghi.rename(columns={'lower': 'city_min_wind', 'upper': 'city_max_wind'}, inplace=True)

# Check for missing values in the Shapefile
missing_values = gdf_city_ghi[['COUNTYNS', 'city_min_wind', 'city_max_wind']].isna().sum()
print(f"Number of missing values in Shapefile:\n{missing_values}")

# Inspect COUNTYNS values in both datasets after cleaning
print("\nCleaned COUNTYNS values in the CSV file:")
print(df_charging_stations['COUNTYNS'].unique())

print("\nCleaned COUNTYNS values in the Shapefile:")
print(gdf_city_ghi['COUNTYNS'].unique())

# Merge the two datasets on the COUNTYNS field
df_final = df_charging_stations.merge(
    gdf_city_ghi[['COUNTYNS', 'city_min_wind', 'city_max_wind']],
    on='COUNTYNS',
    how='left'  # Use left join to ensure all original charging stations are retained
)

# Check for unmatched records
unmatched_final = df_final[df_final['city_min_wind'].isna()]
if not unmatched_final.empty:
    print(f"\nThe following charging stations were not matched with city data:\n{unmatched_final[['Station Name', 'City', 'STATEFP', 'COUNTYFP', 'COUNTYNS']]}")
else:
    print("All charging stations were successfully matched with city data.")

# Verify that the number of rows in the final DataFrame matches the original
if len(df_final) != len(df_charging_stations):
    print("Warning: The number of charging stations in the final output does not match the original data.")
else:
    print("Confirmed: The number of charging stations in the final output matches the original data.")

# Save the updated CSV file
df_final.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
print(f"File has been successfully saved to {output_csv_path}")

# Data quality inspection in USA

In [ ]:
import pandas as pd

# Define file paths
input_csv_path = 'USAmatched_charging_stations_within_city_boundaries_GHIwind_maxmin.csv'
output_csv_path_updated = 'USA_charging_stations_with_updated_min_max_values.csv'

# Read the input CSV file
df_charging_stations = pd.read_csv(input_csv_path)

# Group by city (COUNTYNS) and calculate actual min and max values for GHI and wind
grouped = df_charging_stations.groupby('COUNTYNS').agg(
    actual_min_GHI=('GHI_value', 'min'),
    actual_max_GHI=('GHI_value', 'max'),
    actual_min_wind=('city_min_wind', 'min'),
    actual_max_wind=('city_max_wind', 'max')
).reset_index()

# Create a copy of the original DataFrame to store updated values
df_updated = df_charging_stations.copy()

# Update min and max values in the original DataFrame based on actual station-level data
for index, row in grouped.iterrows():
    city_mask = df_charging_stations['COUNTYNS'] == row['COUNTYNS']
    
    # Update solar (GHI) min and max values if actual values are more extreme
    if df_charging_stations.loc[city_mask, 'GHI_min'].iloc[0] > row['actual_min_GHI']:
        df_updated.loc[city_mask, 'GHI_min'] = row['actual_min_GHI']
    if df_charging_stations.loc[city_mask, 'GHI_max'].iloc[0] < row['actual_max_GHI']:
        df_updated.loc[city_mask, 'GHI_max'] = row['actual_max_GHI']

    # Update wind min and max values if actual values are more extreme
    if df_charging_stations.loc[city_mask, 'city_min_wind'].iloc[0] > row['actual_min_wind']:
        df_updated.loc[city_mask, 'city_min_wind'] = row['actual_min_wind']
    if df_charging_stations.loc[city_mask, 'city_max_wind'].iloc[0] < row['actual_max_wind']:
        df_updated.loc[city_mask, 'city_max_wind'] = row['actual_max_wind']

# Save the updated DataFrame to a new CSV file
df_updated.to_csv(output_csv_path_updated, index=False, encoding='utf-8-sig')

print(f"Updated min/max values have been successfully saved to {output_csv_path_updated}")

# Calculation of matching level indicators in USA

In [ ]:
import pandas as pd

# Define file paths
updated_csv_path = 'USA_charging_stations_with_updated_min_max_values.csv'

# Read the updated CSV file
df_charging_stations = pd.read_csv(updated_csv_path)

# Define wind power density threshold
wind_power_density_threshold = 16.5375

# Calculate PV matching level (Metric 1) for each charging station
df_charging_stations['PV_matching_level'] = (
    (df_charging_stations['GHI_value'] - df_charging_stations['GHI_min']) / 
    (df_charging_stations['GHI_max'] - df_charging_stations['GHI_min'])
)

# Identify charging stations with wind power density above or equal to the threshold
df_charging_stations['is_wind_eligible'] = df_charging_stations['Power_Density'] >= wind_power_density_threshold

# Group by city and filter out stations in cities with fewer than 5 eligible stations
filtered_cities = []
for city, group in df_charging_stations.groupby('COUNTYNS'):
    eligible_group = group[group['is_wind_eligible']]
    
    if len(eligible_group) < 5:
        # Skip cities with fewer than 5 wind-eligible charging stations
        continue
    
    filtered_cities.append(eligible_group)

# Combine data from all qualified cities
df_filtered_charging_stations = pd.concat(filtered_cities, ignore_index=True)

# Calculate wind matching level (Metric 1) for each eligible charging station
df_filtered_charging_stations['wind_matching_level'] = (
    (df_filtered_charging_stations['Power_Density'] - df_filtered_charging_stations['city_min_wind']) / 
    (df_filtered_charging_stations['city_max_wind'] - df_filtered_charging_stations['city_min_wind'])
)

# Calculate average PV matching level per city (Metric 2), station count, and average PV value
city_avg_PV_matching = df_charging_stations.groupby('COUNTYNS').agg(
    avg_PV_matching_level=('PV_matching_level', 'mean'),
    charging_station_count=('PV_matching_level', 'size'),
    avg_PV_value=('GHI_value', 'mean')  # New: compute average PV (GHI) value
).reset_index()

# Filter out cities with fewer than 5 total charging stations
city_avg_PV_matching = city_avg_PV_matching[city_avg_PV_matching['charging_station_count'] >= 5]

# Calculate average wind matching level per city (Metric 2), station count, and average Power Density
city_avg_wind_matching = df_filtered_charging_stations.groupby('COUNTYNS').agg(
    avg_wind_matching_level=('wind_matching_level', 'mean'),
    charging_station_count=('wind_matching_level', 'size'),
    avg_power_density=('Power_Density', 'mean')  # New: compute average Power Density
).reset_index()

# Save individual PV matching levels (Metric 1) to CSV
output_csv_path_1 = 'USA_charging_stations_PV_matching_level_city_based11.csv'
df_charging_stations[['STATEFP', 'COUNTYFP', 'NAME', 'COUNTYNS', 'GHI_value', 'PV_matching_level']].to_csv(output_csv_path_1, index=False, encoding='utf-8-sig')

# Save city-level average PV matching level (Metric 2), station count, and average PV value to CSV
output_csv_path_2 = 'USA_city_avg_PV_matching_level_with_count_city_based_filtered22.csv'
city_avg_PV_matching.to_csv(output_csv_path_2, index=False, encoding='utf-8-sig')

# Save individual wind matching levels (Metric 1) for eligible stations to CSV
output_csv_path_3 = 'USA_charging_stations_wind_matching_level_city_based11.csv'
df_filtered_charging_stations[['STATEFP', 'COUNTYFP', 'NAME', 'COUNTYNS', 'Power_Density', 'wind_matching_level']].to_csv(output_csv_path_3, index=False, encoding='utf-8-sig')

# Save city-level average wind matching level (Metric 2), station count, and average Power Density to CSV
output_csv_path_4 = 'USA_city_avg_wind_matching_level_with_count_city_based_filtered22.csv'
city_avg_wind_matching.to_csv(output_csv_path_4, index=False, encoding='utf-8-sig')

print(f"File successfully saved to {output_csv_path_1}")
print(f"File successfully saved to {output_csv_path_2}")
print(f"File successfully saved to {output_csv_path_3}")
print(f"File successfully saved to {output_csv_path_4}")

# Based on the above description of programming in USA, data from China and Europe can be obtained. Then, integrating the indicators of China, USA and Europe vertically.

In [ ]:
import pandas as pd

# Define file paths
china_csv_path = 'China_city_avg_PV_matching_level_with_count_city_based_filtered222.csv'##Obtain the code similar to the one in USA mentioned above
eu_csv_path = 'EU_city_avg_PV_matching_level_with_count_city_based_filtered22.csv'##Obtain the code similar to the one in USA mentioned above
usa_csv_path = 'USA_city_avg_PV_matching_level_with_count_city_based_filtered22.csv'

# Read the CSV files
df_china = pd.read_csv(china_csv_path)
df_eu = pd.read_csv(eu_csv_path)
df_usa = pd.read_csv(usa_csv_path)

# Add an 'Area' column to identify the region
df_china['Area'] = 'China'
df_eu['Area'] = 'EU'
df_usa['Area'] = 'USA'

# Standardize column names to ensure proper merging
df_china.rename(columns={'city': 'City'}, inplace=True)
df_eu.rename(columns={'NUTS_NAME_right': 'City'}, inplace=True)
df_usa.rename(columns={'COUNTYNS': 'City'}, inplace=True)

# Ensure all DataFrames have the same column order
columns_order = ['City', 'avg_PV_matching_level', 'charging_station_count', 'avg_PV_value', 'Area']
df_china = df_china[columns_order]
df_eu = df_eu[columns_order]
df_usa = df_usa[columns_order]

# Concatenate the three DataFrames
df_combined = pd.concat([df_china, df_eu, df_usa], ignore_index=True)

# Save the combined DataFrame to a CSV file
output_csv_path = 'combined_city_avg_PV_matching_level_with_area22.csv'
df_combined.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"Combined file has been successfully saved to {output_csv_path}")

In [ ]:
import pandas as pd

# Define file paths
china_csv_path = 'China_city_avg_wind_matching_level_with_count_city_based_filtered222.csv'
eu_csv_path = 'EU_city_avg_wind_matching_level_with_count_city_based_filtered22.csv'
usa_csv_path = 'USA_city_avg_wind_matching_level_with_count_city_based_filtered22.csv'

# Read the CSV files
df_china = pd.read_csv(china_csv_path)
df_eu = pd.read_csv(eu_csv_path)
df_usa = pd.read_csv(usa_csv_path)

# Add an 'Area' column to identify the region
df_china['Area'] = 'China'
df_eu['Area'] = 'EU'
df_usa['Area'] = 'USA'

# Standardize column names to ensure proper merging
df_china.rename(columns={'city': 'City'}, inplace=True)
df_eu.rename(columns={'NUTS_NAME_right': 'City'}, inplace=True)
df_usa.rename(columns={'COUNTYNS': 'City'}, inplace=True)

# Ensure all DataFrames have the same column order
columns_order = ['City', 'avg_wind_matching_level', 'charging_station_count', 'avg_power_density', 'Area']
df_china = df_china[columns_order]
df_eu = df_eu[columns_order]
df_usa = df_usa[columns_order]

# Concatenate the three DataFrames
df_combined = pd.concat([df_china, df_eu, df_usa], ignore_index=True)

# Save the combined DataFrame to a CSV file
output_csv_path = 'combined_city_avg_wind_matching_level_with_area22.csv'
df_combined.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"Combined file has been successfully saved to {output_csv_path}")

In [ ]:
import pandas as pd

# Define file paths
ghi_csv_path = 'combined_city_avg_PV_matching_level_with_area22.csv'
wind_csv_path = 'combined_city_avg_wind_matching_level_with_area22.csv'

# Read the CSV files
df_ghi = pd.read_csv(ghi_csv_path)
df_wind = pd.read_csv(wind_csv_path)

# Merge wind data into the solar (GHI) dataset
# Use a left join to ensure all cities from the solar dataset are retained
df_merged = pd.merge(
    df_ghi,  # Solar (GHI) data as the primary table
    df_wind[['City', 'avg_wind_matching_level', 'avg_power_density']],  # Select only relevant wind columns
    on='City',  # Merge on city name
    how='left'  # Left join: keep all cities from the solar dataset
)

# Fill missing wind values with NaN where no matching city data exists
df_merged['avg_wind_matching_level'] = df_merged['avg_wind_matching_level'].fillna(pd.NA)
df_merged['avg_power_density'] = df_merged['avg_power_density'].fillna(pd.NA)

# Display the first few rows of the merged dataset
print(df_merged.head())

# Save the merged dataset to a new CSV file (optional)
df_merged.to_csv('combined_city_PV_wind_matching_level_with_area22.csv', index=False)

# Calculate the wind and solar power generation of EVCS in USA

In [ ]:
import pandas as pd

# Read the CSV file
data_file_path = 'USA_charging_stations_with_updated_min_max_values.csv'
df = pd.read_csv(data_file_path)

# Define constants
swept_area = 2.7  # m^2, swept area of the wind turbine
efficiency = 0.35  # Wind turbine efficiency
hours_per_year = 3000  # Number of hours per year (estimated operational hours)
days_per_year = 365  # Number of days per year
conversion_efficiency = 0.18  # Photovoltaic (PV) panel conversion efficiency (e.g., 18%)
wind_power_density_threshold = 16.5375  # Wind power density threshold in W/m²

# Function to convert wind power density to annual energy generation
def wind_energy_potential(power_density):
    energy_per_year = power_density * swept_area * efficiency * hours_per_year / 1000  # Convert to kWh
    return energy_per_year

# Function to convert GHI (Global Horizontal Irradiance) to annual PV energy generation
def pv_energy_potential(ghi_value):
    energy_per_year = ghi_value * days_per_year * conversion_efficiency  # kWh/m²/year
    return energy_per_year

# Group data by city (COUNTYNS)
grouped = df.groupby('COUNTYNS')

# Initialize an empty list to store results
results = []

# Counter for the number of valid cities (for wind generation only)
valid_city_count_wind = 0

for county, group in grouped:
    # Filter charging stations with wind power density above the threshold
    can_wind_generate_group = group[group['Power_Density'] >= wind_power_density_threshold]
    
    if len(can_wind_generate_group) >= 5:
        # If at least 5 stations meet the wind condition, calculate average wind energy
        total_wind_energy = can_wind_generate_group['Power_Density'].apply(wind_energy_potential).sum()
        avg_wind_energy = total_wind_energy / len(can_wind_generate_group)
        valid_city_count_wind += 1
    else:
        # If fewer than 5 stations meet the condition, set average wind energy to NaN
        avg_wind_energy = float('nan')
    
    # Calculate average PV energy for all stations in the city
    total_pv_energy = group['GHI_value'].apply(pv_energy_potential).sum()
    avg_pv_energy = total_pv_energy / len(group)
    
    results.append({
        'COUNTYNS': county,
        'Avg_Wind_Energy': avg_wind_energy,
        'Avg_PV_Energy': avg_pv_energy
    })

# Convert results to a DataFrame
result_df = pd.DataFrame(results)

# Add the count of valid cities (for wind) as a separate column (broadcasted)
result_df['Valid_City_Count_Wind'] = valid_city_count_wind

# Save the results to a new CSV file
output_file_path = 'USAcity_renewable_energy_potential_filtered.csv'
result_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

# Based on the above description of programming in USA, data from China and Europe can be obtained. Merge the wind and solar energy of the three regions

In [ ]:
import pandas as pd

# Define file paths
eu_file_path = 'EUcity_renewable_energy_potential_filtered.csv'  # Code derived similarly to the USA version
china_file_path = 'Chinacity_renewable_energy_potential_filtered.csv'  # Code derived similarly to the USA version
usa_file_path = 'USAcity_renewable_energy_potential_filtered.csv'

# Read the CSV files
eu_df = pd.read_csv(eu_file_path)
china_df = pd.read_csv(china_file_path)

# Read the USA data and rename the column to standardize
usa_df = pd.read_csv(usa_file_path)
usa_df.rename(columns={'COUNTYNS': 'City'}, inplace=True)

# Concatenate the three DataFrames
combined_df = pd.concat([eu_df, china_df, usa_df], ignore_index=True)

# Save the combined results to a new CSV file
output_file_path = 'combined_renewable_energy_potential.csv'
combined_df.to_csv(output_file_path, index=False)

print(f"Combined results saved to {output_file_path}")

# The energy and matching levels of wind and solar in the three regions are combined

In [ ]:
import pandas as pd

# Define file paths
renewable_energy_file_path = 'combined_renewable_energy_potential.csv'
coverage_level_file_path = 'combined_city_PV_wind_matching_level_with_area22.csv'

# Read the CSV files
renewable_energy_df = pd.read_csv(renewable_energy_file_path)
coverage_level_df = pd.read_csv(coverage_level_file_path)

# Merge the two DataFrames horizontally based on the 'City' column
merged_df = pd.merge(renewable_energy_df, coverage_level_df, on='City', how='outer')

# Save the merged result to a new CSV file
output_file_path = 'combineddata_windsolarenergy_and_matchinglevel.csv'
merged_df.to_csv(output_file_path, index=False)

print(f"Merged results saved to {output_file_path}")